In [97]:
import numpy as np
import sys
import os
import pandas as pd

# Adjust the path to point to external/AlphaPEM
sys.path.append(os.path.abspath("../external/AlphaPEM"))
# Importing constants' value and functions
from configuration.settings import current_density_parameters, physical_parameters, \
                                   computing_parameters, operating_inputs
from model.AlphaPEM import AlphaPEM

In [98]:
def build_fixed_parameters():
    type_current="polarization"
    type_fuel_cell="EH-31_2.0"
    t_step, i_step, delta_pola, i_EIS, ratio_EIS, f_EIS, t_EIS, current_density = current_density_parameters(type_current)
    # Operating conditions
    *_, i_max_pola = operating_inputs(type_fuel_cell)
    
    # Physical parameters
    Hcl, epsilon_mc, tau, Hmem, Hgdl, epsilon_gdl, epsilon_c, Hgc, Wgc, Lgc, Aact, e, Re, i0_c_ref, kappa_co, \
        kappa_c, a_slim, b_slim, a_switch, C_scl = physical_parameters(type_fuel_cell)
    # Computing parameters
    max_step, n_gdl, t_purge = computing_parameters(type_current, Hgdl, Hcl)

    return {
        "t_step": t_step,
        "i_step": i_step,
        "delta_pola": delta_pola,
        "i_EIS": i_EIS,
        "ratio_EIS": ratio_EIS,
        "f_EIS": f_EIS,
        "t_EIS": t_EIS,
        "current_density": current_density,
        "max_step": max_step,
        "n_gdl": n_gdl,
        "t_purge": t_purge,
        "type_fuel_cell": "manual_setup", 
        "type_current": "polarization",
        "type_auxiliary": "no_auxiliary",
        "type_control": "no_control",
        "type_purge": "no_purge",
        "type_display": "no_display",
        "type_plot": "fixed",
        "C_scl": C_scl,
        "i_max_pola": i_max_pola,
        "Aact": Aact,
        "Hgdl": Hgdl,
        "Hmem": Hmem,
        "Hcl": Hcl,
        "Hgc": Hgc,
        "Wgc": Wgc,
        "Lgc": Lgc
    }

In [99]:
parameter_ranges = {
    # Operating conditions
    "Tfc": (333, 363),  # Cell temperature (K)
    "Pa_des": (1.1e5, 3e5),  # Desired cell pressure (bara)
    "Pc_des": None,  # Desired cell pressure (bara)
    "Sa": (1.1, 3),  # Stoichiometry anode
    "Sc": (1.1, 10),  # Stoichiometry cathode
    "Phi_a_des": (0.1, 1),  # Desired entrance humidity anode
    "Phi_c_des": (0.1, 1),  # Desired entrance humidity cathode

    # Undetermined physical parameters
    "epsilon_gdl": (0.55, 0.8),  # GDL porosity
    "tau": (1.0, 4.0),  # Pore structure coefficient
    "epsilon_mc": (0.15, 0.4),  # CL volume fraction of ionomer
    "epsilon_c": (0.15, 0.3),  # GDL compression ratio
    "e": [3, 4, 5],  # Capillary exponent (should be an integer)
    "Re": (5e-7, 5e-6),  # Electron conduction resistance (Ω·m²)
    "i0_c_ref": (0.001, 500),  # Cathode exchange current density (A/m²)
    "kappa_co": (0.01, 40),  # Crossover correction coefficient (mol/(m·s·Pa))
    "kappa_c": (0, 100),  # Overpotential correction exponent
    "a_slim": (0, 1),  # Slim coefficient a
    "b_slim": (0, 1),  # Slim coefficient b
    "a_switch": (0, 1),  # Slim switch parameter
}

def sample_parameters(n_samples=100, parameter_ranges=parameter_ranges):
    samples = {}
    
    for key, val in parameter_ranges.items():
        if key == 'Pa_des':
            low, high = val
            samples['Pa_des'] = np.random.uniform(low, high, n_samples)
            low, high = (np.maximum(1, samples['Pa_des'] - 0.5), np.maximum(1, samples['Pa_des'] - 0.1))
            samples['Pc_des'] = np.random.uniform(low, high)

        elif isinstance(val, tuple):  # Continuous range
            low, high = val
            samples[key] = np.random.uniform(low, high, n_samples)

        elif isinstance(val, list):  # Discrete
            samples[key] = np.random.choice(val, n_samples)
        
    return samples

print(sample_parameters(n_samples=1))
sampled_parameters = sample_parameters(n_samples=1)
fixed_parameters = build_fixed_parameters()
combined_parameters = {**sampled_parameters, **fixed_parameters}

{'Tfc': array([339.12493568]), 'Pa_des': array([222125.9853271]), 'Pc_des': array([222125.82062018]), 'Sa': array([1.23949121]), 'Sc': array([7.33330767]), 'Phi_a_des': array([0.95398867]), 'Phi_c_des': array([0.41801043]), 'epsilon_gdl': array([0.77430655]), 'tau': array([1.15191775]), 'epsilon_mc': array([0.18375994]), 'epsilon_c': array([0.25304676]), 'e': array([3]), 'Re': array([3.75040494e-06]), 'i0_c_ref': array([319.08309997]), 'kappa_co': array([27.33067524]), 'kappa_c': array([71.19168649]), 'a_slim': array([0.73572008]), 'b_slim': array([0.30487856]), 'a_switch': array([0.06690778])}


In [100]:
def convert_arrays_to_numbers(d):
    new_dict = {}
    for key, value in d.items():
        new_dict[key] = float(value)
    return new_dict

sampled_parameters = convert_arrays_to_numbers(sampled_parameters)
combined_parameters = {**sampled_parameters, **fixed_parameters}

C:\Users\dejvi\AppData\Local\Temp\ipykernel_9576\562995038.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_dict[key] = float(value)


In [101]:
Simulator = AlphaPEM(**combined_parameters)

In [102]:
def plot_polarisation_curve(variables, operating_inputs, parameters):
    # Extraction of the variables
    t, Ucell_t = np.array(variables['t']), np.array(variables['Ucell'])
    # Extraction of the operating inputs and the parameters
    current_density = operating_inputs['current_density']
    t_step, i_step, i_max_pola = parameters['t_step'], parameters['i_step'], parameters['i_max_pola']
    delta_pola = parameters['delta_pola']
    i_EIS, t_EIS, f_EIS = parameters['i_EIS'], parameters['t_EIS'], parameters['f_EIS']
    type_fuel_cell, type_auxiliary = parameters['type_fuel_cell'], parameters['type_auxiliary']
    type_control, type_plot = parameters['type_control'], parameters['type_plot']

    if type_plot == "fixed":
        # Creation of ifc_t
        n = len(t)
        ifc_t = np.zeros(n)
        for i in range(n):
            ifc_t[i] = current_density(t[i], parameters) / 1e4  # Conversion in A/cm²

        # Recovery of ifc and Ucell from the model after each stack stabilisation
        delta_t_load_pola, delta_t_break_pola, delta_i_pola, delta_t_ini_pola = delta_pola
        nb_loads = int(i_max_pola / delta_i_pola + 1)  # Number of loads which are made
        ifc_discretized = np.zeros(nb_loads)
        Ucell_discretized = np.zeros(nb_loads)
        for i in range(nb_loads):
            t_load = delta_t_ini_pola + (i + 1) * (delta_t_load_pola + delta_t_break_pola) - delta_t_break_pola / 10
            #                                                                                    # time for measurement
            idx = (np.abs(t - t_load)).argmin()  # the corresponding index
            ifc_discretized[i] = ifc_t[idx]  # the last value at the end of each load
            Ucell_discretized[i] = Ucell_t[idx]  # the last value at the end of each load

    return {'ifc': ifc_discretized, 'Ucell': Ucell_discretized}

pola_curve_data = plot_polarisation_curve(Simulator.variables, Simulator.operating_inputs, Simulator.parameters)

pola_curve_data = pd.DataFrame(data)
pola_curve_data

,ifc,Ucell
0,0.000815,8.442044
1,0.100813,7.430610
2,0.200813,6.668241
3,0.300813,6.023677
4,0.400813,5.460122
5,0.500813,4.957777
6,0.600813,4.504377
7,0.700813,4.091706
8,0.800813,3.713965
9,0.900813,3.366903
